## Type safety / hints
See [doc](https://beam.apache.org/documentation/sdks/python-type-safety/)

In [7]:
import typing

from apache_beam import coders
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
p = beam.Pipeline(InteractiveRunner())

In [ ]:
@beam.typehints.with_output_types(int)
@beam.typehints.with_input_types(int)
class FilterEvensDoFn(beam.DoFn):
    def process(self, element):
        if element % 2 == 0:
            yield element

def get_even(element: int) -> int:
    if element % 2 == 0:
        return element

e1 = (
    p
    | beam.Create(['1', '2', '3'])
    | beam.ParDo(FilterEvensDoFn())
)

e2 = (
    p
    | beam.Create(['one','two','three']) 
    | beam.Filter(lambda x: x % 2 == 0).with_input_types(int) 
)

ib.show(e1)
ib.show(e2)

In [18]:
class Employee(typing.NamedTuple):
    id: str
    name: str

coders.registry.register_coder(Employee, coders.RowCoder)
    
# class Employee:
#   def __init__(self, id, name):
#     self.id = str
#     self.name = name

# class EmployeeCoder(beam.coders.Coder):
#   def encode(self, employee):
#     return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

#   def decode(self, s):
#     return Employee(*s.decode('utf-8').split(':'))

#   def is_deterministic(self):
#     return True

# beam.coders.registry.register_coder(Employee, EmployeeCoder)

def split_file(input):
  id, name, dep_id = input.split(',')[0], input.split(',')[1], input.split(',')[2]
  return Employee(id, name), int(dep_id)

result = (
    p
	| beam.io.ReadFromText('dept_data.txt')
    | beam.Map(split_file)
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
	)
ib.show(result, n=5)